# Week 4: Model Optimization and Ablation Studies

**Project:** US Collisions (2016-2023) Forecasting  
**Student:** Mario Cuevas  
**Submission:** Project Update 4  

---

## Overview

This notebook contains Week 4 deliverables:
1. Hyperparameter optimization using Optuna
2. Ablation studies (Weather vs. Time features)
3. State-level vs. National model comparison
4. Final model performance analysis
5. Comprehensive results summary

**Prerequisites:** Trained models from Week 3 (GRU, LSTM, TCN, Transformer) and baseline models from Week 2 (Random Forest, Prophet).

---

## Setup and Dependencies

In [ ]:
!pip install -q optuna pandas numpy matplotlib seaborn plotly scikit-learn tensorflow

print("Packages installed successfully!")

Packages installed successfully!


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam

# Optimization
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

warnings.filterwarnings('ignore')
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported!")
print(f"TensorFlow: {tf.__version__}")
print(f"Optuna: {optuna.__version__}")

Libraries imported!
TensorFlow: 2.19.0
Optuna: 4.6.0


## Load Preprocessed Data

Load the feature-engineered dataset created in Week 2.

In [ ]:
# Load preprocessed data
daily_features = pd.read_csv('/content/daily_accidents_features.csv')
daily_features['Date'] = pd.to_datetime(daily_features['Date'])

daily_features['Month'] = daily_features['Date'].dt.month
daily_features['DayOfWeek'] = daily_features['Date'].dt.dayofweek
daily_features['DayOfYear'] = daily_features['Date'].dt.dayofyear
daily_features['Quarter'] = daily_features['Date'].dt.quarter

print(f"Data loaded: {daily_features.shape}")
print(f"Date range: {daily_features['Date'].min()} to {daily_features['Date'].max()}")
daily_features.head()

Data loaded: (2568, 27)
Date range: 2016-01-14 00:00:00 to 2023-03-31 00:00:00


,Date,Accident_Count,Avg_Severity,Avg_Temperature,Avg_Humidity,Avg_Visibility,Avg_Wind_Speed,Avg_Precipitation,Accident_Count_7d_MA,Accident_Count_30d_MA,...,Avg_Humidity_Lag_1,Avg_Visibility_Lag_1,Avg_Temperature_Lag_7,Avg_Humidity_Lag_7,Avg_Visibility_Lag_7,Weather_Risk_Index,Month,DayOfWeek,DayOfYear,Quarter
0,2016-01-14,7,4.000000,31.000000,69.000000,10.000000,3.000000,0.000000,7.00,7.00,...,NaN,NaN,NaN,NaN,NaN,-1.252220,1,3,14,1
1,2016-02-08,60,2.316667,35.388333,91.816667,7.130000,6.681667,0.015000,33.50,33.50,...,69.000000,10.000000,NaN,NaN,NaN,1.527727,2,0,39,1
2,2016-02-09,59,2.542373,23.532203,87.118644,2.410169,11.461017,0.005932,42.00,42.00,...,91.816667,7.130000,NaN,NaN,NaN,3.596967,2,1,40,1
3,2016-02-10,49,2.510204,17.765306,77.571429,3.875510,12.767347,0.005306,43.75,43.75,...,87.118644,2.410169,NaN,NaN,NaN,2.900429,2,2,41,1
4,2016-02-11,93,2.354839,15.888172,65.311828,9.263441,8.975269,0.007204,53.60,53.60,...,77.571429,3.875510,NaN,NaN,NaN,-0.015010,2,3,42,1


## Prepare Data for Modeling

In [ ]:
weather_features = ['Avg_Temperature', 'Avg_Humidity', 'Avg_Visibility', 'Avg_Wind_Speed', 'Avg_Precipitation'] # Corrected names and removed 'Pressure'
time_features = ['Month', 'DayOfWeek', 'DayOfYear', 'Quarter',
                 'Accident_Count_Lag_1', 'Accident_Count_Lag_7', 'Accident_Count_Lag_14', 'Accident_Count_Lag_30',
                 'Accident_Count_7d_MA', 'Accident_Count_30d_MA'] # Corrected MA names and removed STD names
all_features = weather_features + time_features

X = daily_features[all_features]
y = daily_features['Accident_Count']

split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
dates_train, dates_test = daily_features['Date'][:split_idx], daily_features['Date'][split_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"\nWeather features: {len(weather_features)}")
print(f"Time features: {len(time_features)}")
print(f"Total features: {len(all_features)}")

Training samples: 2054
Test samples: 514

Weather features: 5
Time features: 10
Total features: 15


In [ ]:
def create_sequences(data, target, lookback=30, forecast_horizon=1):
    X_seq, y_seq = [], []
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X_seq.append(data[i:i+lookback])
        y_seq.append(target[i+lookback+forecast_horizon-1])
    return np.array(X_seq), np.array(y_seq)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

X_train_scaled = X_scaled[:split_idx]
X_test_scaled = X_scaled[split_idx:]
y_train_scaled = y_scaled[:split_idx]
y_test_scaled = y_scaled[split_idx:]

LOOKBACK = 30
FORECAST_HORIZON = 1

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, LOOKBACK, FORECAST_HORIZON)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, LOOKBACK, FORECAST_HORIZON)

print(f"\nSequence data prepared:")
print(f"X_train_seq: {X_train_seq.shape}")
print(f"X_test_seq: {X_test_seq.shape}")


Sequence data prepared:
X_train_seq: (2024, 30, 15)
X_test_seq: (484, 30, 15)


---

# 1. Ablation Study: Weather vs Time Features

Compare model performance using different feature subsets to understand feature importance.

In [ ]:
print("="*60)
print("ABLATION STUDY 1: WEATHER VS TIME FEATURES")
print("="*60)
print("\nTesting Random Forest with different feature combinations...")

ABLATION STUDY 1: WEATHER VS TIME FEATURES

Testing Random Forest with different feature combinations...


In [ ]:
# Model 1: Weather Features Only
print("\n1. Training with Weather Features Only...")
rf_weather = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_weather.fit(X_train[weather_features], y_train)
pred_weather = rf_weather.predict(X_test[weather_features])

mae_weather = mean_absolute_error(y_test, pred_weather)
r2_weather = r2_score(y_test, pred_weather)
rmse_weather = np.sqrt(mean_squared_error(y_test, pred_weather))

print(f"   MAE: {mae_weather:.2f}")
print(f"   RMSE: {rmse_weather:.2f}")
print(f"   R²: {r2_weather:.4f}")


1. Training with Weather Features Only...
   MAE: 1264.16
   RMSE: 1555.41
   R²: -0.2320


In [ ]:
# Model 2: Time Features Only
print("\n2. Training with Time Features Only...")
rf_time = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_time.fit(X_train[time_features], y_train)
pred_time = rf_time.predict(X_test[time_features])

mae_time = mean_absolute_error(y_test, pred_time)
r2_time = r2_score(y_test, pred_time)
rmse_time = np.sqrt(mean_squared_error(y_test, pred_time))

print(f"   MAE: {mae_time:.2f}")
print(f"   RMSE: {rmse_time:.2f}")
print(f"   R²: {r2_time:.4f}")


2. Training with Time Features Only...
   MAE: 574.07
   RMSE: 862.96
   R²: 0.6208


In [ ]:
# Model 3: Combined Features (Baseline from Week 2)
print("\n3. Training with Combined Features...")
rf_combined = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_combined.fit(X_train[all_features], y_train)
pred_combined = rf_combined.predict(X_test[all_features])

mae_combined = mean_absolute_error(y_test, pred_combined)
r2_combined = r2_score(y_test, pred_combined)
rmse_combined = np.sqrt(mean_squared_error(y_test, pred_combined))

print(f"   MAE: {mae_combined:.2f}")
print(f"   RMSE: {rmse_combined:.2f}")
print(f"   R²: {r2_combined:.4f}")


3. Training with Combined Features...
   MAE: 568.17
   RMSE: 817.95
   R²: 0.6593


In [ ]:
ablation_results = pd.DataFrame({
    'Feature Set': ['Weather Only', 'Time Only', 'Combined'],
    'MAE': [mae_weather, mae_time, mae_combined],
    'RMSE': [rmse_weather, rmse_time, rmse_combined],
    'R²': [r2_weather, r2_time, r2_combined],
    'Num Features': [len(weather_features), len(time_features), len(all_features)]
})

print("\n" + "="*60)
print("ABLATION STUDY RESULTS")
print("="*60)
print(ablation_results.to_string(index=False))
print("="*60)


ABLATION STUDY RESULTS
 Feature Set         MAE        RMSE        R²  Num Features
Weather Only 1264.163091 1555.412858 -0.232015             5
   Time Only  574.065171  862.962660  0.620765            10
    Combined  568.170680  817.947614  0.659297            15


In [ ]:
# Visualize ablation study results
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('MAE Comparison', 'RMSE Comparison', 'R² Comparison')
)

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

fig.add_trace(
    go.Bar(x=ablation_results['Feature Set'], y=ablation_results['MAE'],
           marker_color=colors, showlegend=False),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=ablation_results['Feature Set'], y=ablation_results['RMSE'],
           marker_color=colors, showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=ablation_results['Feature Set'], y=ablation_results['R²'],
           marker_color=colors, showlegend=False),
    row=1, col=3
)

fig.update_layout(height=400, title_text="Ablation Study: Feature Set Impact")
fig.update_xaxes(tickangle=-45)
fig.show()

In [ ]:
# Calculate percentage contribution
weather_contribution = (r2_weather / r2_combined) * 100
time_contribution = (r2_time / r2_combined) * 100

print(f"\nFeature Set Contributions:")
print(f"Weather features contribute: {weather_contribution:.1f}% of combined model performance")
print(f"Time features contribute: {time_contribution:.1f}% of combined model performance")
print(f"\nConclusion: {'Time' if r2_time > r2_weather else 'Weather'} features are more predictive")


Feature Set Contributions:
Weather features contribute: -35.2% of combined model performance
Time features contribute: 94.2% of combined model performance

Conclusion: Time features are more predictive


---

# 2. Ablation Study: State-Level vs National Model

Compare performance of a national aggregated model vs. state-specific models.

In [ ]:
print("="*60)
print("ABLATION STUDY 2: STATE-LEVEL VS NATIONAL MODEL")
print("="*60)
print("\nNote: This analysis requires state-specific data.")
print("Loading original dataset to extract state information...")

ABLATION STUDY 2: STATE-LEVEL VS NATIONAL MODEL

Note: This analysis requires state-specific data.
Loading original dataset to extract state information...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load original dataset with state information
# Note: This may take time with large dataset
df_original = pd.read_csv('/content/drive/MyDrive/US_Accidents_March23.csv',
                          usecols=['Start_Time', 'State', 'Severity'])
df_original['Start_Time'] = pd.to_datetime(df_original['Start_Time'], errors='coerce')
df_original['Date'] = df_original['Start_Time'].dt.date

print(f"Original data loaded: {df_original.shape}")

Original data loaded: (7728394, 4)


In [ ]:
# Identify top 5 states by accident count
top_states = df_original['State'].value_counts().head(5).index.tolist()
print(f"\nTop 5 states: {top_states}")

# Aggregate by state and date
state_daily = df_original.groupby(['State', 'Date']).size().reset_index(name='Accident_Count')
state_daily['Date'] = pd.to_datetime(state_daily['Date'])

print(f"State-level daily data created: {state_daily.shape}")


Top 5 states: ['CA', 'FL', 'TX', 'SC', 'NY']
State-level daily data created: (101230, 3)


In [ ]:
# Train state-specific models
state_results = {}

for state in top_states:
    print(f"\nTraining model for {state}...")

    # Get state data
    state_data = state_daily[state_daily['State'] == state].sort_values('Date').reset_index(drop=True)

    state_data['Month'] = state_data['Date'].dt.month
    state_data['DayOfWeek'] = state_data['Date'].dt.dayofweek
    state_data['Lag_1'] = state_data['Accident_Count'].shift(1)
    state_data['Lag_7'] = state_data['Accident_Count'].shift(7)
    state_data['MA_7'] = state_data['Accident_Count'].rolling(window=7).mean()
    state_data = state_data.dropna()

    state_split = int(len(state_data) * 0.8)
    state_features = ['Month', 'DayOfWeek', 'Lag_1', 'Lag_7', 'MA_7']

    X_state_train = state_data[state_features][:state_split]
    y_state_train = state_data['Accident_Count'][:state_split]
    X_state_test = state_data[state_features][state_split:]
    y_state_test = state_data['Accident_Count'][state_split:]

    state_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    state_model.fit(X_state_train, y_state_train)
    state_pred = state_model.predict(X_state_test)

    state_mae = mean_absolute_error(y_state_test, state_pred)
    state_r2 = r2_score(y_state_test, state_pred)

    state_results[state] = {
        'MAE': state_mae,
        'R²': state_r2,
        'Samples': len(state_data),
        'Avg_Accidents': state_data['Accident_Count'].mean()
    }

    print(f"   {state}: MAE={state_mae:.2f}, R²={state_r2:.4f}")


Training model for CA...
   CA: MAE=128.05, R²=0.4955

Training model for FL...
   FL: MAE=123.11, R²=0.7388

Training model for TX...
   TX: MAE=46.34, R²=0.5834

Training model for SC...
   SC: MAE=44.28, R²=0.4816

Training model for NY...
   NY: MAE=41.42, R²=0.5051


In [ ]:
state_comparison = pd.DataFrame(state_results).T
state_comparison['Model'] = 'State-Specific'
state_comparison.reset_index(inplace=True)
state_comparison.rename(columns={'index': 'State'}, inplace=True)

national_row = pd.DataFrame({
    'State': ['National'],
    'MAE': [mae_combined],
    'R²': [r2_combined],
    'Samples': [len(daily_features)],
    'Avg_Accidents': [daily_features['Accident_Count'].mean()],
    'Model': ['National']
})

comparison_df = pd.concat([state_comparison, national_row], ignore_index=True)

print("\n" + "="*60)
print("STATE-LEVEL VS NATIONAL MODEL COMPARISON")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)


STATE-LEVEL VS NATIONAL MODEL COMPARISON
   State        MAE       R²  Samples  Avg_Accidents          Model
      CA 128.050976 0.495510   2494.0     627.325180 State-Specific
      FL 123.106479 0.738826   2401.0     318.457309 State-Specific
      TX  46.342684 0.583425   2402.0     226.338468 State-Specific
      SC  44.280668 0.481582   2370.0     148.075527 State-Specific
      NY  41.424982 0.505098   2483.0     124.908981 State-Specific
National 568.170680 0.659297   2568.0    2554.914720       National


In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('MAE by Model', 'R² by Model'))

fig.add_trace(
    go.Bar(x=comparison_df['State'], y=comparison_df['MAE'],
           marker_color=['lightcoral']*len(top_states) + ['steelblue'],
           showlegend=False),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=comparison_df['State'], y=comparison_df['R²'],
           marker_color=['lightcoral']*len(top_states) + ['steelblue'],
           showlegend=False),
    row=1, col=2
)

fig.update_layout(height=400, title_text="State-Specific vs National Model Performance")
fig.update_xaxes(tickangle=-45)
fig.show()

---

# 3. Hyperparameter Optimization with Optuna

Systematic hyperparameter tuning for the best-performing model (GRU).

In [ ]:
print("="*60)
print("HYPERPARAMETER OPTIMIZATION: GRU MODEL")
print("="*60)
print("\nUsing Optuna to find optimal hyperparameters...")

HYPERPARAMETER OPTIMIZATION: GRU MODEL

Using Optuna to find optimal hyperparameters...


In [ ]:
def build_gru_model(input_shape, units=64, dropout=0.2, learning_rate=0.001):
    model = models.Sequential([
        layers.GRU(units, return_sequences=True, input_shape=input_shape),
        layers.Dropout(dropout),
        layers.GRU(units//2, return_sequences=False),
        layers.Dropout(dropout),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    return model

In [ ]:
# Define Optuna objective function
def objective(trial):
    tf.keras.backend.clear_session()

    units = trial.suggest_categorical('units', [32, 64, 96])
    dropout = trial.suggest_float('dropout', 0.2, 0.4)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)  # Lower max
    batch_size = trial.suggest_categorical('batch_size', [32, 64])

    try:
        # Build model
        model = build_gru_model(
            input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
            units=units,
            dropout=dropout,
            learning_rate=learning_rate
        )

        # Early stopping with more patience
        early_stop = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            min_delta=1e-4
        )

        # Reduce LR on plateau
        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6
        )

        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=20,  # Reduced epochs for speed
            batch_size=batch_size,
            validation_split=0.2,
            callbacks=[early_stop, reduce_lr],
            verbose=0
        )

        # Get best validation loss
        val_loss = min(history.history['val_loss'])

        if np.isnan(val_loss) or np.isinf(val_loss):
            return float('inf')

        return val_loss

    except Exception as e:
        print(f"Trial failed with error: {e}")
        return float('inf')

print("Updated objective function defined!")

Updated objective function defined!


In [ ]:

print("Checking for NaN values...")
print(f"X_train_seq has NaN: {np.isnan(X_train_seq).any()}")
print(f"y_train_seq has NaN: {np.isnan(y_train_seq).any()}")
print(f"X_test_seq has NaN: {np.isnan(X_test_seq).any()}")
print(f"y_test_seq has NaN: {np.isnan(y_test_seq).any()}")

# Remove any NaN values
print(f"\nBefore cleaning: {X_train_seq.shape}")

# Find rows with NaN
nan_mask_train = ~np.isnan(X_train_seq).any(axis=(1,2)) & ~np.isnan(y_train_seq)
nan_mask_test = ~np.isnan(X_test_seq).any(axis=(1,2)) & ~np.isnan(y_test_seq)

# Filter out NaN rows
X_train_seq = X_train_seq[nan_mask_train]
y_train_seq = y_train_seq[nan_mask_train]
X_test_seq = X_test_seq[nan_mask_test]
y_test_seq = y_test_seq[nan_mask_test]

print(f"After cleaning: {X_train_seq.shape}")
print(f"Removed {(~nan_mask_train).sum()} training samples with NaN")

Checking for NaN values...
X_train_seq has NaN: True
y_train_seq has NaN: False
X_test_seq has NaN: False
y_test_seq has NaN: False

Before cleaning: (2024, 30, 15)
After cleaning: (1994, 30, 15)
Removed 30 training samples with NaN


In [ ]:
# Run Optuna optimization
print("Starting optimization (this may take 10-20 minutes)...\n")

study = optuna.create_study(direction='minimize', study_name='GRU_Optimization')
study.optimize(objective, n_trials=20, show_progress_bar=True)

print("\nOptimization complete!")

[I 2025-11-22 21:24:50,496] A new study created in memory with name: GRU_Optimization


Starting optimization (this may take 10-20 minutes)...



  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-11-22 21:24:57,604] Trial 0 finished with value: 0.0390801839530468 and parameters: {'units': 32, 'dropout': 0.2678731127109153, 'learning_rate': 0.00021178811702341228, 'batch_size': 32}. Best is trial 0 with value: 0.0390801839530468.
[I 2025-11-22 21:25:05,252] Trial 1 finished with value: 0.025115543976426125 and parameters: {'units': 64, 'dropout': 0.21635822622224388, 'learning_rate': 0.0001299063677936705, 'batch_size': 32}. Best is trial 1 with value: 0.025115543976426125.
[I 2025-11-22 21:25:14,127] Trial 2 finished with value: 0.03066902607679367 and parameters: {'units': 32, 'dropout': 0.32162398172994694, 'learning_rate': 0.00016472374060467598, 'batch_size': 32}. Best is trial 1 with value: 0.025115543976426125.
[I 2025-11-22 21:25:27,164] Trial 3 finished with value: 0.022595809772610664 and parameters: {'units': 32, 'dropout': 0.2163587658837665, 'learning_rate': 0.00041345457560162816, 'batch_size': 32}. Best is trial 3 with value: 0.022595809772610664.
[I 2025-

In [ ]:
print("\n" + "="*60)
print("BEST HYPERPARAMETERS")
print("="*60)
print(f"Best validation loss: {study.best_value:.6f}")
print("\nOptimal hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")
print("="*60)


BEST HYPERPARAMETERS
Best validation loss: 0.015145

Optimal hyperparameters:
  units: 96
  dropout: 0.2026439980301298
  learning_rate: 0.0006333667355395976
  batch_size: 32


In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(title="Optuna Optimization History", height=400)
fig.show()

In [ ]:
# Visualize parameter importances
fig = optuna.visualization.plot_param_importances(study)
fig.update_layout(title="Hyperparameter Importance", height=400)
fig.show()

In [ ]:
# Train final model with best hyperparameters
print("\nTraining final model with optimal hyperparameters...")

best_model = build_gru_model(
    input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
    units=study.best_params['units'],
    dropout=study.best_params['dropout'],
    learning_rate=study.best_params['learning_rate']
)

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = best_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=study.best_params['batch_size'],
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

print("\nFinal model trained!")


Training final model with optimal hyperparameters...
Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0958 - mae: 0.2419 - val_loss: 0.0308 - val_mae: 0.1361
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0203 - mae: 0.1198 - val_loss: 0.0273 - val_mae: 0.1317
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0179 - mae: 0.1117 - val_loss: 0.0243 - val_mae: 0.1245
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0162 - mae: 0.1048 - val_loss: 0.0235 - val_mae: 0.1205
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0142 - mae: 0.0983 - val_loss: 0.0235 - val_mae: 0.1179
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0134 - mae: 0.0950 - val_loss: 0.0232 - val_mae: 0.1173
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0120 - mae: 0.0898 - val_loss: 0.0213 - val_mae: 0.1129
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0109 - mae: 0.0845 - val_loss: 0.0207 - val_mae: 0.1093
Epoch 9/50
50/50

In [79]:
optimized_pred_scaled = best_model.predict(X_test_seq).flatten()
optimized_pred = scaler_y.inverse_transform(optimized_pred_scaled.reshape(-1, 1)).flatten()
optimized_actual = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

optimized_mae = mean_absolute_error(optimized_actual, optimized_pred)
optimized_r2 = r2_score(optimized_actual, optimized_pred)

print(f"Optimized GRU - MAE: {optimized_mae:.2f}, R²: {optimized_r2:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Optimized GRU - MAE: 790.12, R²: 0.3387


---

# 4. Final Comprehensive Results

Summary of all models and optimizations from Week 2-4.

In [80]:
final_results = pd.DataFrame({
    'Model': [
        'Random Forest (Baseline)',
        'Prophet (Baseline)',
        'GRU (Week 3)',
        'LSTM + Attention (Week 3)',
        'TCN (Week 3)',
        'Transformer (Week 3)',
        'GRU Optimized (Week 4)'
    ],
    'MAE': [
        mae_combined,
        1500,             # Prophet
        1247,             # GRU
        1305,             # LSTM
        1389,             # TCN
        1654,             # Transformer
        optimized_mae     #Week 4 optimization (750)
    ],
    'R²': [
        r2_combined,
        0.320,
        gru_r2,
        lstm_r2,
        tcn_r2,
        transformer_r2,
        optimized_r2     # From Week 4
    ],
    'Week': ['Week 2', 'Week 2', 'Week 3', 'Week 3', 'Week 3', 'Week 3', 'Week 4']
})

final_results = final_results.sort_values('R²', ascending=False)

print("\n" + "="*70)
print("COMPREHENSIVE MODEL COMPARISON (WEEKS 2-4)")
print("="*70)
print(final_results.to_string(index=False))
print("="*70)




COMPREHENSIVE MODEL COMPARISON (WEEKS 2-4)
                    Model         MAE       R²   Week
 Random Forest (Baseline)  568.170680 0.659297 Week 2
             GRU (Week 3) 1247.000000 0.415358 Week 3
   GRU Optimized (Week 4)  790.116297 0.338654 Week 4
       Prophet (Baseline) 1500.000000 0.320000 Week 2
LSTM + Attention (Week 3) 1305.000000 0.296821 Week 3
             TCN (Week 3) 1389.000000 0.201178 Week 3
     Transformer (Week 3) 1654.000000 0.026973 Week 3


In [82]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('MAE Comparison', 'R² Comparison')
)

# Color by week
colors = ['blue' if w == 'Week 2' else 'green' if w == 'Week 3' else 'red'
          for w in final_results['Week']]

fig.add_trace(
    go.Bar(x=final_results['Model'], y=final_results['MAE'],
           marker_color=colors, showlegend=False),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=final_results['Model'], y=final_results['R²'],
           marker_color=colors, showlegend=False),
    row=1, col=2
)

fig.update_layout(
    height=500,
    title_text="Final Model Performance Comparison (Weeks 2-4)"
)
fig.update_xaxes(tickangle=-45)
fig.show()

In [83]:
baseline_r2 = final_results[final_results['Model'] == 'Random Forest (Baseline)']['R²'].values[0]
best_dl_r2 = final_results[final_results['Week'].isin(['Week 3', 'Week 4'])]['R²'].max()
improvement = ((best_dl_r2 / baseline_r2) - 1) * 100

print("\n" + "="*60)
print("KEY FINDINGS")
print("="*60)
print(f"1. Best Model: {final_results.iloc[0]['Model']}")
print(f"   - R²: {final_results.iloc[0]['R²']:.4f}")
print(f"   - MAE: {final_results.iloc[0]['MAE']:.2f}")
print(f"\n2. Deep Learning Performance:")
print(f"   - Achieved {(best_dl_r2/baseline_r2)*100:.1f}% of baseline performance")
print(f"\n3. Feature Importance:")
print(f"   - Time features: {(r2_time/r2_combined)*100:.1f}% contribution")
print(f"   - Weather features: {(r2_weather/r2_combined)*100:.1f}% contribution")
print(f"\n4. Optimization Impact:")
print(f"   - Hyperparameter tuning improved validation loss")
print(f"   - Most important parameter: {list(study.best_params.keys())[0]}")
print("="*60)


KEY FINDINGS
1. Best Model: Random Forest (Baseline)
   - R²: 0.6593
   - MAE: 568.17

2. Deep Learning Performance:
   - Achieved 63.0% of baseline performance

3. Feature Importance:
   - Time features: 94.2% contribution
   - Weather features: -35.2% contribution

4. Optimization Impact:
   - Hyperparameter tuning improved validation loss
   - Most important parameter: units


In [84]:

import os
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("Saving all results to files...")
print("="*60)

ablation_results.to_csv('ablation_study_results.csv', index=False)
print("✓ Saved: ablation_study_results.csv")

if 'comparison_df' in locals():
    comparison_df.to_csv('state_vs_national_comparison.csv', index=False)
    print("✓ Saved: state_vs_national_comparison.csv")

optuna_results = pd.DataFrame({
    'Trial': range(len(study.trials)),
    'Value': [trial.value for trial in study.trials],
    'Params': [str(trial.params) for trial in study.trials]
})
optuna_results.to_csv('optuna_optimization_results.csv', index=False)
print("✓ Saved: optuna_optimization_results.csv")

best_params_df = pd.DataFrame([study.best_params])
best_params_df.to_csv('optuna_best_params.csv', index=False)
print("✓ Saved: optuna_best_params.csv")

final_results.to_csv('final_model_comparison.csv', index=False)
print("✓ Saved: final_model_comparison.csv")

optimized_results = pd.DataFrame({
    'Metric': ['MAE', 'RMSE', 'R²'],
    'Value': [optimized_mae, optimized_rmse, optimized_r2]
})
optimized_results.to_csv('optimized_model_performance.csv', index=False)
print("✓ Saved: optimized_model_performance.csv")

print("="*60)
print(f"All CSV files saved successfully!")


Saving all results to files...
✓ Saved: ablation_study_results.csv
✓ Saved: state_vs_national_comparison.csv
✓ Saved: optuna_optimization_results.csv
✓ Saved: optuna_best_params.csv
✓ Saved: final_model_comparison.csv
✓ Saved: optimized_model_performance.csv
All CSV files saved successfully!
